In [70]:
from pyspark.sql import SparkSession
from operator import add

In [71]:
spark = SparkSession.builder.appName("Assig1_PageRank").getOrCreate()

In [72]:
fname = "./web-BerkStan.txt"
data = spark.read.text(fname).rdd.map(lambda r: r[0])

In [73]:
def extractIDs(x):
    temp = x.split("\t")
    return temp

def getValidIDs(x):
    if len(x) == 2:
        return x[0].isdigit() and x[1].isdigit()
    return False

def calc(page2list, rank):
    n = len(page2list)
    for page2 in page2list:
        yield (page2, rank / n)

In [74]:
linkinfo = data.map(lambda x : extractIDs(x)).filter(lambda x : getValidIDs(x)).groupByKey()#.cache()
ranks = linkinfo.map(lambda url_neighbors : (url_neighbors[0], 1.0))

n_iterations = 10
for _ in range(n_iterations):
    contributions = linkinfo.join(ranks).flatMap(
          lambda page1_page2list_rank1 : calc(page1_page2list_rank1[1][0], page1_page2list_rank1[1][1]))
    ranks = contributions.reduceByKey(add).mapValues(lambda contrib : 0.15 + (0.85 * contrib))
    
ranks.take(10)
        


[(u'287144', 0.1872392009999748),
 (u'282424', 0.47561981964639233),
 (u'496975', 2.9531547274362135),
 (u'623634', 0.16634540504803774),
 (u'186829', 0.2579919395127193),
 (u'311016', 0.2346604361923148),
 (u'432407', 0.6082312015910813),
 (u'670977', 0.32450802940722284),
 (u'103549', 0.9321981726804518),
 (u'576354', 0.6389431562485941)]